# Building a regression model for predicting house sale prices

In [1]:
import pickle
import pathlib

import numpy as np
import pandas as pd

In [2]:
DATA_DIR = pathlib.Path.cwd() / 'data'
print(DATA_DIR)

/Users/enriccogemha/Developer/HousePriceRegressor/data


In [3]:
clean_data_path = DATA_DIR / 'processed' / 'ames_model_data.pkl'

In [4]:
with open(clean_data_path, 'rb') as file:
    model_data = pickle.load(file)

In [5]:
model_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2877 entries, 0 to 2929
Columns: 165 entries, Lot.Frontage to Exterior_Other
dtypes: bool(119), float64(34), int64(12)
memory usage: 1.4 MB


### Justificativa para manter grande parte da feature engineering anterior e opções para explorar

lorem ipsus

## Train-test splitting

In [6]:
X = model_data.drop(columns=['SalePrice']).copy()
y = model_data['SalePrice'].copy()

In [7]:
X, y

(      Lot.Frontage  Lot.Area  Lot.Shape  Land.Slope  Overall.Qual  \
 0            141.0   31770.0          1           0             5   
 1             80.0   11622.0          0           0             4   
 2             81.0   14267.0          1           0             5   
 3             93.0   11160.0          0           0             6   
 4             74.0   13830.0          1           0             4   
 ...            ...       ...        ...         ...           ...   
 2925          37.0    7937.0          1           0             5   
 2926          68.0    8885.0          1           1             4   
 2927          62.0   10441.0          0           0             4   
 2928          77.0   10010.0          0           1             4   
 2929          74.0    9627.0          0           1             6   
 
       Overall.Cond  Mas.Vnr.Area  Exter.Qual  Exter.Cond  BsmtFin.SF.1  ...  \
 0                4         112.0           2           2         639.0  ...  

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
RANDOM_SEED = 42  # Any number here, really.

In [ ]:
Xtrain, Xtest, ytrain, ytest = train_test_split(
    X,
    y,
    test_size=0.25,
    random_state=RANDOM_SEED,
)


In [ ]:
X.shape, Xtrain.shape, Xtest.shape

In [ ]:
y.shape, ytrain.shape, ytest.shape

## Primeiro teste: modelo linear com scaling e cross-validation

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer


float64_columns = Xtrain.select_dtypes('float64').columns

col = ColumnTransformer(
    [
        ('scale', StandardScaler(), float64_columns),
    ],
    remainder='passthrough',
)

col.fit(Xtrain)

Xtrain_scaled = col.transform(Xtrain)
Xtest_scaled = col.transform(Xtest)


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error

lr = LinearRegression()

grid = {
    'fit_intercept': [True, False],
}

grid_search = GridSearchCV(
    lr,
    grid,
    cv=5,
    scoring='neg_mean_squared_error',
    return_train_score=True,
)

In [ ]:
grid_search.fit(Xtrain_scaled, ytrain)

print(f'Best parameters: {grid_search.best_params_}')
print(f'Best score: {grid_search.best_score_}')

rmse = np.sqrt(-grid_search.best_score_)
error_percent = 100 * (10**rmse - 1)
print(f'Average error is {error_percent:.2f}%')

Best parameters: {'fit_intercept': False}
Best score: -14571779528729.12
Average error is inf%


/var/folders/x3/bbk8nj6s67s9nv5gvt6q6gxh0000gn/T/ipykernel_75770/1903569813.py:7: RuntimeWarning: overflow encountered in scalar power
  error_percent = 100 * (10**rmse - 1)


In [30]:
val = grid_search.best_estimator_.predict(Xtest_scaled)
rmse = np.sqrt(mean_squared_error(ytest, val))
error_percent = 100 * (10**rmse - 1)
print(f'Average error is {error_percent:.2f}%')

Average error is inf%


/var/folders/x3/bbk8nj6s67s9nv5gvt6q6gxh0000gn/T/ipykernel_75770/1112581313.py:3: RuntimeWarning: overflow encountered in scalar power
  error_percent = 100 * (10**rmse - 1)


## Segundo experimento: Modelo Lasso e scaling

In [ ]:
from sklearn.linear_model import Lasso

grid = {
    'alpha': np.logspace(-8, -3, 200),
}

lasso = Lasso()

grid_search = GridSearchCV(
    lasso,
    grid,
    cv=5,
    scoring='neg_mean_squared_error',
    return_train_score=True,
)

In [ ]:
grid_search.fit(Xtrain_scaled, ytrain)

print(f'Best parameters: {grid_search.best_params_}')
print(f'Best score: {grid_search.best_score_}')

/Users/enriccogemha/Developer/HousePriceRegressor/venv/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.354e-01, tolerance: 5.147e-03
  model = cd_fast.enet_coordinate_descent(
/Users/enriccogemha/Developer/HousePriceRegressor/venv/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.160e+00, tolerance: 4.972e-03
  model = cd_fast.enet_coordinate_descent(
/Users/enriccogemha/Developer/HousePriceRegressor/venv/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the numbe

Best parameters: {'alpha': 8.804883581643464e-05}
Best score: -0.003296308851327881


In [ ]:
rmse = np.sqrt(-grid_search.best_score_)
error_percent = 100 * (10**rmse - 1)
print(f'Average error is {error_percent:.2f}%')

Percebe-se que Lasso tem menor erro que o linear

## Terceiro experimento: Lasso + fazer transformação de log para features com skewness (assimetria)

Transformação de log é conhecida por fazer uma feature que tem distribuição assimétrica ter uma distribuição mais próxima da normal

In [35]:
#test skewness for all columns and get index of those with skewness > 0.5
from scipy.stats import skew

skewness = Xtrain.select_dtypes(np.number).apply(skew)

skewness

Lot.Frontage        1.374507
Lot.Area           13.935570
Lot.Shape           1.260202
Land.Slope          5.096601
Overall.Qual        0.238328
Overall.Cond        0.631834
Mas.Vnr.Area        2.584621
Exter.Qual         -0.813090
Exter.Cond         -1.319756
BsmtFin.SF.1        1.013526
BsmtFin.SF.2        4.222772
Bsmt.Unf.SF         0.955611
Total.Bsmt.SF       0.729935
Heating.QC          0.546224
Electrical          4.168022
X1st.Flr.SF         1.263409
X2nd.Flr.SF         0.842831
Low.Qual.Fin.SF    11.186456
Gr.Liv.Area         1.096900
Bsmt.Full.Bath      0.614981
Bsmt.Half.Bath      3.940715
Full.Bath           0.159638
Half.Bath           0.686145
Bedroom.AbvGr       0.408400
Kitchen.AbvGr       4.018729
Kitchen.Qual       -0.456881
TotRms.AbvGrd       0.770448
Functional          5.199897
Fireplaces          0.730059
Garage.Cars        -0.179943
Garage.Area         0.264373
Paved.Drive         3.172562
Wood.Deck.SF        1.974402
Open.Porch.SF       2.676164
Enclosed.Porch

In [36]:
skewness = skewness[abs(skewness) > 3]

skew_features = Xtrain[skewness.index]

skew_features.columns

Index(['Lot.Area', 'Land.Slope', 'BsmtFin.SF.2', 'Electrical',
       'Low.Qual.Fin.SF', 'Bsmt.Half.Bath', 'Kitchen.AbvGr', 'Functional',
       'Paved.Drive', 'Enclosed.Porch', 'X3Ssn.Porch', 'Screen.Porch',
       'Pool.Area', 'Misc.Val'],
      dtype='object')

In [37]:
def log_tf(feature):
    return np.log1p(feature)

Xtrain_skew = Xtrain.copy()
Xtest_skew = Xtest.copy()

Xtrain_skew[skew_features.columns] = Xtrain_skew[skew_features.columns].apply(log_tf)
Xtest_skew[skew_features.columns] = Xtest_skew[skew_features.columns].apply(log_tf)


In [38]:
col.fit(Xtrain_skew)

Xtrain_scaled = col.transform(Xtrain_skew)
Xtest_scaled = col.transform(Xtest_skew)

In [39]:
grid_search.fit(Xtrain_scaled, ytrain)

/Users/enriccogemha/Developer/HousePriceRegressor/venv/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.677e+00, tolerance: 5.147e-03
  model = cd_fast.enet_coordinate_descent(
/Users/enriccogemha/Developer/HousePriceRegressor/venv/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.594e+00, tolerance: 4.972e-03
  model = cd_fast.enet_coordinate_descent(
/Users/enriccogemha/Developer/HousePriceRegressor/venv/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the numbe

GridSearchCV(cv=5, estimator=Lasso(),
             param_grid={'alpha': array([1.00000000e-08, 1.05956018e-08, 1.12266777e-08, 1.18953407e-08,
       1.26038293e-08, 1.33545156e-08, 1.41499130e-08, 1.49926843e-08,
       1.58856513e-08, 1.68318035e-08, 1.78343088e-08, 1.88965234e-08,
       2.00220037e-08, 2.12145178e-08, 2.24780583e-08, 2.38168555e-08,
       2.52353917e-08, 2.67384162e-08, 2.83309610e-08, 3....
       3.33129479e-04, 3.52970730e-04, 3.73993730e-04, 3.96268864e-04,
       4.19870708e-04, 4.44878283e-04, 4.71375313e-04, 4.99450512e-04,
       5.29197874e-04, 5.60716994e-04, 5.94113398e-04, 6.29498899e-04,
       6.66991966e-04, 7.06718127e-04, 7.48810386e-04, 7.93409667e-04,
       8.40665289e-04, 8.90735464e-04, 9.43787828e-04, 1.00000000e-03])},
             return_train_score=True, scoring='neg_mean_squared_error')

In [ ]:
rmse = np.sqrt(-grid_search.best_score_)
error_percent = 100 * (10**rmse - 1)
print(f'Average error is {error_percent:.2f}%')

Average error is 12.89%


In [ ]:
val = grid_search.best_estimator_.predict(Xtest_scaled)
rmse = np.sqrt(mean_squared_error(ytest, val))
error_percent = 100 * (10**rmse - 1)
print(f'Average error is {error_percent:.2f}%')

Average error is 15.04%


Percebe-se uma melhora com esse feature engineering

## Quarto experimento : Elastic net

Elastic net é um modelo que combina a regularização ridge e Lasso

In [ ]:
from sklearn.linear_model import ElasticNet

grid = {
    'alpha': np.logspace(-8, -3, 10),
    'l1_ratio': np.linspace(0.01, 1, 50),
}

elastic = ElasticNet()

grid_search = GridSearchCV(
    elastic,
    grid,
    cv=5,
    scoring='neg_mean_squared_error',
    return_train_score=True,
)

In [ ]:
grid_search.fit(Xtrain_scaled, ytrain)

print(f'Best parameters: {grid_search.best_params_}')
print(f'Best score: {grid_search.best_score_}')

In [ ]:
rmse = np.sqrt(-grid_search.best_score_)
error_percent = 100 * (10**rmse - 1)
print(f'Average error is {error_percent:.2f}%')

## Considerando todas as opções, Elastic net foi a que teve menor erro

#### Teste final

In [ ]:
val = grid_search.best_estimator_.predict(Xtest_scaled)
rmse = np.sqrt(mean_squared_error(ytest, val))
error_percent = 100 * (10**rmse - 1)
print(f'Average error is {error_percent:.2f}%')